In [1]:
import zipfile
import os
import shutil # 디렉토리 삭제 등 고수준 파일 작업용

# 테스트용 파일 생성
test_files_to_zip = ['file1.txt', 'file2.txt', 'subdir/file3.txt']
#subdir 생성
os.makedirs('subdir', exist_ok=True)
for filename in test_files_to_zip:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"이 파일은 '{filename}' 입니다.\n")

zip_filename = 'my_archive.zip'

# 1. ZIP 파일 생성 및 파일 추가
try:
    # 'w' 모드로 열고, ZIP_DEFLATED 압축 사용
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in test_files_to_zip:
            # zipf.write(파일경로, 아카이브내경로)
            zipf.write(file_path, arcname=os.path.basename(file_path)) # 아카이브에는 파일명만 저장
            print(f"ZIP에 추가: '{file_path}' -> '{os.path.basename(file_path)}'")
    print(f"\n'{zip_filename}' 생성 완료.")

    # 2. ZIP 파일 내용 확인
    print(f"\n--- '{zip_filename}' 내용 ---")
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        # zipf.namelist() 로 파일 목록 얻기
        print(f"파일 목록: {zipf.namelist()}")
        
        # 각 파일의 정보 확인
        for zipinfo in zipf.filelist:
            print(f"  파일: {zipinfo.filename}, 크기: {zipinfo.file_size}, 압축 크기: {zipinfo.compress_size}")
            # 압축률 계산 (0이면 압축 안됨)
            if zipinfo.file_size > 0 and zipinfo.compress_size > 0:
                ratio = (1 - zipinfo.compress_size / zipinfo.file_size) * 100
                print(f"  압축률: {ratio:.1f}%")

    # 3. ZIP 내 특정 파일 읽기
    print("\n--- ZIP 내 'file1.txt' 읽기 ---")
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        # zipf.open()으로 멤버 파일 객체 얻기
        with zipf.open('file1.txt') as member_file:
            content = member_file.read().decode('utf-8')
            print(content.strip()) # 앞뒤 공백 제거하고 출력

    # 4. ZIP 파일 압축 해제
    extract_dir = 'unzipped_files'
    print(f"\n--- '{zip_filename}' 압축 해제 시작 ---")
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        zipf.extractall(path=extract_dir) # 지정된 경로에 모두 해제
        print(f"'{extract_dir}' 디렉토리에 압축 해제 완료.")
        print(f"해제된 파일 목록: {os.listdir(extract_dir)}")

finally:
    # 생성된 파일 및 디렉토리 정리
    for f in test_files_to_zip:
        if os.path.exists(f): os.remove(f)
    if os.path.exists('subdir'): shutil.rmtree('subdir')
    if os.path.exists(zip_filename): os.remove(zip_filename)
    if os.path.exists(extract_dir): shutil.rmtree(extract_dir)
    print("\n테스트 파일 및 결과 정리 완료.")


ZIP에 추가: 'file1.txt' -> 'file1.txt'
ZIP에 추가: 'file2.txt' -> 'file2.txt'
ZIP에 추가: 'subdir/file3.txt' -> 'file3.txt'

'my_archive.zip' 생성 완료.

--- 'my_archive.zip' 내용 ---
파일 목록: ['file1.txt', 'file2.txt', 'file3.txt']
  파일: file1.txt, 크기: 37, 압축 크기: 40
  압축률: -8.1%
  파일: file2.txt, 크기: 37, 압축 크기: 40
  압축률: -8.1%
  파일: file3.txt, 크기: 44, 압축 크기: 47
  압축률: -6.8%

--- ZIP 내 'file1.txt' 읽기 ---
이 파일은 'file1.txt' 입니다.

--- 'my_archive.zip' 압축 해제 시작 ---
'unzipped_files' 디렉토리에 압축 해제 완료.
해제된 파일 목록: ['file1.txt', 'file3.txt', 'file2.txt']

테스트 파일 및 결과 정리 완료.
